In [1]:
from pytube import YouTube
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from IPython.display import clear_output
import os, glob, random, time, ffmpy
import speech_recognition as sr
import wave, contextlib
from google.cloud import speech
from pydub import AudioSegment
from pydub.silence import split_on_silence
from tensorflow.keras.preprocessing.text import Tokenizer

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
links = []

In [3]:
driver = webdriver.Chrome('G:\My Drive\Desktop\Tools\chromedriver89.exe');
driver.get('https://www.youtube.com/c/TED/videos'); time.sleep(3);
#driver.get('https://www.youtube.com/c/TED/videos?view=0&sort=da&flow=grid'); time.sleep(3)

page = []

while len(page)<1:
    driver.find_element_by_css_selector("div#search-input > input#search").send_keys(Keys.PAGE_DOWN);
    page.append(driver.find_elements_by_xpath("/html//div[@id='details']/div[@id='meta']//a[@href]"));
    clear_output()
    print(f'Total pages scraped: {len(page)}');
for x in page[-1]:
    if x.get_attribute('href') not in links:
        links.append(x.get_attribute('href'))
    else:
        pass
driver.close()
driver.quit()

Total pages scraped: 1


In [4]:
pd.DataFrame(links).to_csv(f'TEDx_links.csv', index = False)
data = pd.read_csv(f'TEDx_links.csv')
data.head()

,0
0,https://www.youtube.com/watch?v=Qm02X0aE8uU
1,https://www.youtube.com/watch?v=Axh07mJ9Ag4
2,https://www.youtube.com/watch?v=Mkelhs_OVMc
3,https://www.youtube.com/watch?v=g4xGbbDACDw
4,https://www.youtube.com/watch?v=CpzjPnDW1G4


In [5]:
str(data.loc[0]).split(' ')[4].split('\n')[0] 
#This feels too long just to query the value at index as string instead of object.

'https://www.youtube.com/watch?v=Qm02X0aE8uU'

In [6]:
print(f'Total links: {len(data)}')

Total links: 30


In [7]:
#Remove old files to make a clean batch
for file in [x for x in os.listdir('rawFiles')]:
    os.remove(f'rawFiles\\{file}')

#Download videos from YouTube using scraped link list
for x in range(3):
    yt = YouTube(str(data.loc[x]).split(' ')[4].split('\n')[0])
    yt.streams.filter(only_audio=True)[0].download('rawFiles')
    clear_output()
    print(f'Downloaded {x+1} videos.')

#Convert .mp4 to .wav and encoding filename for processing
for idx, file in enumerate([x for x in os.listdir('rawFiles') if x.split('.')[-1]=='mp4']):
    ff = ffmpy.FFmpeg(executable="drivers\\ffmpeg.exe",
                      inputs = {f"rawFiles\\{file}" : None},
                      outputs = {f"rawFiles\\{idx}.wav" : f'-acodec pcm_s16le -ac 1 -ar 16000'})
    ff.run()
    os.remove(f'rawFiles\\{file}')
print('All videos converted.')

Downloaded 3 videos.
All videos converted.


In [8]:
#Method1: Splitting by a fixed time length.
for file in [x for x in os.listdir('splitFiles')]:
    os.remove(f'splitFiles\\{file}')

cut_len = 10 #seconds

for idx, file in enumerate([x for x in os.listdir('rawFiles')]):
    with contextlib.closing(wave.open(f'rawFiles\\{file}','r')) as f: #opening the file to read
        duration = round(f.getnframes() / float(f.getframerate())) #total frames / the framerate
    x=0
    while x < (duration/cut_len): #doing it this way to have x as 1 or could have simply added cut_len to x at each iter
        ff = ffmpy.FFmpeg(executable="drivers\\ffmpeg.exe",\
                          inputs = {f"rawFiles\\{file}" : None},\
                          outputs = {f"splitFiles\\{idx}_{x}.wav" : f'-ss {x*cut_len} -t {cut_len}'}) # -ss(starting at) -t(seconds to keep) that 12 is to crop out TEDx intro sound :P
        ff.run()
        x+=1

In [9]:
#Method2: Splitting by silence in audio.

'''for file in [x for x in os.listdir('rawFiles') if x.split('.')[-1]=='wav']:
    song = AudioSegment.from_wav(f"rawFiles\\{file}")
    chunks = split_on_silence(song, min_silence_len = 800, silence_thresh = -50)
    for idx, chunk in enumerate(chunks):
        chunk.export(f"splitFiles\\{file}_{idx}.wav", format='wav')'''

'for file in [x for x in os.listdir(\'rawFiles\') if x.split(\'.\')[-1]==\'wav\']:\n    song = AudioSegment.from_wav(f"rawFiles\\{file}")\n    chunks = split_on_silence(song, min_silence_len = 800, silence_thresh = -50)\n    for idx, chunk in enumerate(chunks):\n        chunk.export(f"splitFiles\\{file}_{idx}.wav", format=\'wav\')'

In [23]:
files = [x for x in os.listdir('splitFiles')]

transcript = []

r = sr.Recognizer()

for file in files:
    temp=[]
    audio = sr.AudioFile(f'splitFiles\\{file}')
    with audio as source:
        audio_file = r.record(source)
        try:
            temp = (r.recognize_google(audio_file))
            transcript.append(temp)
            pd.DataFrame(transcript).to_csv(f'TEDx_Lines.csv', index = False)
        except:
            pass
print(len(transcript))
print(transcript[0:50])

In [757]:
transcript[random.randint(0, len(transcript))]

"of 55 countries is getting easier and faster by the day so it won't be long before none effect"

In [765]:
#Tensorflow tokenizer to create tokens of sentences by encoding words into int categoricals
tokens = Tokenizer(num_words = 100)
tokens.fit_on_texts(transcript)
print(tokens.word_index)

ng': 715, 'roshan': 716, 'perspectives': 717, 'microbes': 718, 'assemble': 719, 'interdisciplinary': 720, 'teams': 721, 'shot': 722, 'chasing': 723, 'famous': 724, 'collaborated': 725, 'engineers': 726, 'robots': 727, 'gentle': 728, 'stressors': 729, 'jellyfish': 730, 'radcliffe': 731, 'institute': 732, 'spicy': 733, 'innova': 734, 'crysta': 735, 'ise': 736, 'papaya': 737, 'cents': 738, 'universe': 739, 'songs': 740, 'future': 741, 'began': 742, 'discussing': 743, 'facebook': 744, 'anytime': 745, 'profibus': 746, 'tracking': 747, 'intraspecies': 748, 'code': 749, "didn't": 750, 'sindhi': 751, 'lada': 752, 'report': 753, 'analysing': 754, 'communicate': 755, 'sequences': 756, 'regional': 757, 'dialects': 758, 'british': 759, 'powerful': 760, 'analyse': 761, 'map': 762, 'meaning': 763, 'scientists': 764, 'unknown': 765, 'stone': 766, 'dimensional': 767, 'updated': 768, 'researchers': 769, 'selected': 770, 'annotated': 771, 'thousand': 772, 'sperm': 773, 'boneta': 774, 'millions': 775, 'a